In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import itertools
from sklearn.model_selection import KFold
import networkx as nx
import matplotlib.pyplot as plt

# Create example data
np.random.seed(42)
n = 100  # Number of samples
data = {
    'y1': np.random.rand(n) * 100,
    'y2': np.random.rand(n) * 50,
    'x1': np.random.rand(n) * 10,
    'x2': np.random.rand(n) * 20,
    'x3': np.random.rand(n) * 30,
    'weights': np.random.rand(n) + 1  # Weights for WLS
}

df_example = pd.DataFrame(data)

# Replace some values with NaNs to test missing data handling
df_example.loc[5:10, 'x1'] = np.nan
df_example.loc[15, 'y1'] = np.inf

# Functions provided earlier
def plot_dag_from_residuals(residuals, title):
    G = nx.DiGraph()
    
    for col in residuals.columns:
        for idx in residuals.index:
            if residuals.at[idx, col] != 0:
                G.add_edge(idx, col, weight=residuals.at[idx, col])
    
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=3000, font_size=10)
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    plt.title(title)
    plt.show()

def get_residuals(df, method='ols', weights=None):
    residuals = {}
    for y_var in df.columns:
        if y_var == 'weights':
            continue
        X_vars = list(df.columns)
        X_vars.remove(y_var)
        X_vars.remove('weights')
        X = df[X_vars].copy()
        X["Constant"] = 1  # Add constant term
        y = df[y_var]
        
        # Remove rows with NaN or infinite values
        combined = pd.concat([X, y], axis=1)
        combined = combined.replace([np.inf, -np.inf], np.nan).dropna()
        X = combined[X_vars + ["Constant"]]
        y = combined[y_var]
        
        if method == 'wls' and weights is not None:
            model = sm.WLS(y, X, weights=weights)
        else:
            model = sm.OLS(y, X)
            
        results = model.fit()
        residuals["$\\epsilon_{" + y_var + "}$"] = results.resid
    
    return pd.DataFrame(residuals)

def run_regression_combinations_kfold(df, dependent_var, independent_vars, always_include=None, never_include=None, 
                                      df_name='df', include_constant=False, n_splits=10, random_state=None, method='ols', weights=None):
    np.random.seed(random_state)
    df = df.replace([np.inf, -np.inf], np.nan).dropna()

    y = df[dependent_var]

    results = []

    if always_include is None:
        always_include = []
    if never_include is None:
        never_include = []

    independent_vars = [var for var in independent_vars if var not in never_include]

    all_residuals = pd.DataFrame(index=df.index)

    for i in range(1, len(independent_vars) + 1):
        for combo in itertools.combinations(independent_vars, i):
            combo = list(always_include) + list(combo)
            X = df[combo]

            if include_constant:
                X = sm.add_constant(X)
                combo = ['const'] + combo

            kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
            mse_list = []
            r_squared_list = []
            beta_estimates = []
            residuals_list = []

            for train_index, test_index in kf.split(X):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                if method == 'wls' and weights is not None:
                    model = sm.WLS(y_train, X_train, weights=weights.iloc[train_index])
                else:
                    model = sm.OLS(y_train, X_train)
                    
                result = model.fit()
                y_pred = result.predict(X_test)
                mse = np.mean((y_test - y_pred) ** 2)
                mse_list.append(mse)
                r_squared = result.rsquared
                r_squared_list.append(r_squared)
                beta_estimates.append(result.params)
                residuals = y_test - y_pred
                residuals_list.append(residuals)

            top_3_mse = sorted(mse_list)[:3]
            avg_top_3_mse = np.mean(top_3_mse)
            avg_beta_estimates = np.mean(beta_estimates, axis=0)
            avg_r_squared = np.mean(r_squared_list)
            avg_residuals = pd.concat(residuals_list).groupby(level=0).mean()
            avg_residuals.name = '_'.join(combo)  # Assign a name to the Series

            all_residuals = all_residuals.join(avg_residuals, rsuffix=f'_{"_".join(combo)}')

            result = {
                'DataFrame': df_name,
                'Model': ', '.join(combo),
                'r-squared': avg_r_squared,
                'avg_top_3_mse': avg_top_3_mse,
                'Variables': '<br>'.join(
                    [f'{combo[idx]}: {avg_beta_estimates[idx]:.4f}' for idx in range(len(combo))])
            }
            for idx, var in enumerate(combo):
                result[var] = avg_beta_estimates[idx]
            results.append(result)

    results_df = pd.DataFrame(results)
    
    return results_df

def plot_dags_for_residuals(df_dict, method='ols', weights=None):
    for df_name, df in df_dict.items():
        residuals_df = get_residuals(df, method=method, weights=weights)
        plot_dag_from_residuals(residuals_df, f'DAG of Residuals for {df_name}')

In [3]:
# Example usage with the example data
# Define dependent and independent variables
dependent_var = 'y1'
independent_vars = ['x1', 'x2', 'x3']
weights = df_example['weights']

# Run regression combinations using OLS
results_ols = run_regression_combinations_kfold(df_example, dependent_var, independent_vars, 
                                                df_name='Example OLS', include_constant=True, n_splits=5, random_state=42, method='ols')

# Display the results
print(results_ols)

     DataFrame              Model  r-squared  avg_top_3_mse  \
0  Example OLS          const, x1   0.010591     813.679519   
1  Example OLS          const, x2   0.040777     755.465316   
2  Example OLS          const, x3   0.026255     789.280759   
3  Example OLS      const, x1, x2   0.059823     749.891617   
4  Example OLS      const, x1, x3   0.036289     829.754830   
5  Example OLS      const, x2, x3   0.076741     762.326777   
6  Example OLS  const, x1, x2, x3   0.096496     772.839379   

                                           Variables      const        x1  \
0                      const: 52.3655<br>x1: -0.8554  52.365503 -0.855444   
1                      const: 57.2425<br>x2: -0.9701  57.242548       NaN   
2                       const: 40.2703<br>x3: 0.4802  40.270277       NaN   
3       const: 65.1277<br>x1: -1.2546<br>x2: -1.0894  65.127667 -1.254621   
4        const: 44.7775<br>x1: -0.8308<br>x3: 0.4774  44.777488 -0.830808   
5        const: 49.4582<br>x2: -1

In [4]:
# Create a dictionary of dataframes
df_dict = {'Example OLS': df_example}

In [5]:
# Plotting DAGs for the residuals
plot_dags_for_residuals(df_dict, method='ols')

In [7]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import itertools
from sklearn.model_selection import KFold
import networkx as nx
import matplotlib.pyplot as plt

# Create example data
np.random.seed(42)
n = 100  # Number of samples
data = {
    'y1': np.random.rand(n) * 100,
    'y2': np.random.rand(n) * 50,
    'x1': np.random.rand(n) * 10,
    'x2': np.random.rand(n) * 20,
    'x3': np.random.rand(n) * 30,
    'weights': np.random.rand(n) + 1  # Weights for WLS
}

df_example = pd.DataFrame(data)

# Replace some values with NaNs to test missing data handling
df_example.loc[5:10, 'x1'] = np.nan
df_example.loc[15, 'y1'] = np.inf

# Function to plot DAG from residuals
def plot_dag_from_residuals(residuals, title):
    G = nx.DiGraph()
    
    for col in residuals.columns:
        for idx in residuals.index:
            if residuals.at[idx, col] != 0:
                G.add_edge(idx, col, weight=residuals.at[idx, col])
    
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=3000, font_size=10)
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    plt.title(title)
    plt.show()

# Function to get residuals from regression
def get_residuals(df, method='ols', weights=None):
    residuals = {}
    for y_var in df.columns:
        if y_var == 'weights':
            continue
        X_vars = list(df.columns)
        X_vars.remove(y_var)
        X_vars.remove('weights')
        X = df[X_vars].copy()
        X["Constant"] = 1  # Add constant term
        y = df[y_var]
        
        # Remove rows with NaN or infinite values
        combined = pd.concat([X, y], axis=1)
        combined = combined.replace([np.inf, -np.inf], np.nan).dropna()
        X = combined[X_vars + ["Constant"]]
        y = combined[y_var]
        
        if method == 'wls' and weights is not None:
            model = sm.WLS(y, X, weights=weights)
        else:
            model = sm.OLS(y, X)
            
        results = model.fit()
        residuals["$\\epsilon_{" + y_var + "}$"] = results.resid
    
    return pd.DataFrame(residuals)

# Function to run regression combinations with k-fold cross-validation
def run_regression_combinations_kfold(df, dependent_var, independent_vars, always_include=None, never_include=None, 
                                      df_name='df', include_constant=False, n_splits=10, random_state=None, method='ols', weights=None):
    np.random.seed(random_state)
    df = df.replace([np.inf, -np.inf], np.nan).dropna()

    y = df[dependent_var]

    results = []

    if always_include is None:
        always_include = []
    if never_include is None:
        never_include = []

    independent_vars = [var for var in independent_vars if var not in never_include]

    all_residuals = pd.DataFrame(index=df.index)

    for i in range(1, len(independent_vars) + 1):
        for combo in itertools.combinations(independent_vars, i):
            combo = list(always_include) + list(combo)
            X = df[combo]

            if include_constant:
                X = sm.add_constant(X)
                combo = ['const'] + combo

            kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
            mse_list = []
            r_squared_list = []
            beta_estimates = []
            residuals_list = []

            for train_index, test_index in kf.split(X):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                if method == 'wls' and weights is not None:
                    model = sm.WLS(y_train, X_train, weights=weights.iloc[train_index])
                else:
                    model = sm.OLS(y_train, X_train)
                    
                result = model.fit()
                y_pred = result.predict(X_test)
                mse = np.mean((y_test - y_pred) ** 2)
                mse_list.append(mse)
                r_squared = result.rsquared
                r_squared_list.append(r_squared)
                beta_estimates.append(result.params)
                residuals = y_test - y_pred
                residuals_list.append(residuals)

            top_3_mse = sorted(mse_list)[:3]
            avg_top_3_mse = np.mean(top_3_mse)
            avg_beta_estimates = np.mean(beta_estimates, axis=0)
            avg_r_squared = np.mean(r_squared_list)
            avg_residuals = pd.concat(residuals_list).groupby(level=0).mean()
            avg_residuals.name = '_'.join(combo)  # Assign a name to the Series

            all_residuals = all_residuals.join(avg_residuals, rsuffix=f'_{"_".join(combo)}')

            result = {
                'DataFrame': df_name,
                'Model': ', '.join(combo),
                'r-squared': avg_r_squared,
                'avg_top_3_mse': avg_top_3_mse,
                'Variables': '<br>'.join(
                    [f'{combo[idx]}: {avg_beta_estimates[idx]:.4f}' for idx in range(len(combo))])
            }
            for idx, var in enumerate(combo):
                result[var] = avg_beta_estimates[idx]
            results.append(result)

    results_df = pd.DataFrame(results)
    
    return results_df

# Function to plot DAGs for residuals
def plot_dags_for_residuals(df_dict, method='ols', weights=None):
    for df_name, df in df_dict.items():
        residuals_df = get_residuals(df, method=method, weights=weights)
        plot_dag_from_residuals(residuals_df, f'DAG of Residuals for {df_name}')


In [8]:
# Example usage with the example data
# Define dependent and independent variables
dependent_var = 'y1'
independent_vars = ['x1', 'x2', 'x3']
weights = df_example['weights']

# Run regression combinations using OLS
results_ols = run_regression_combinations_kfold(df_example, dependent_var, independent_vars, 
                                                df_name='Example OLS', include_constant=True, n_splits=5, random_state=42, method='ols')

# Display the results
print(results_ols)

     DataFrame              Model  r-squared  avg_top_3_mse  \
0  Example OLS          const, x1   0.010591     813.679519   
1  Example OLS          const, x2   0.040777     755.465316   
2  Example OLS          const, x3   0.026255     789.280759   
3  Example OLS      const, x1, x2   0.059823     749.891617   
4  Example OLS      const, x1, x3   0.036289     829.754830   
5  Example OLS      const, x2, x3   0.076741     762.326777   
6  Example OLS  const, x1, x2, x3   0.096496     772.839379   

                                           Variables      const        x1  \
0                      const: 52.3655<br>x1: -0.8554  52.365503 -0.855444   
1                      const: 57.2425<br>x2: -0.9701  57.242548       NaN   
2                       const: 40.2703<br>x3: 0.4802  40.270277       NaN   
3       const: 65.1277<br>x1: -1.2546<br>x2: -1.0894  65.127667 -1.254621   
4        const: 44.7775<br>x1: -0.8308<br>x3: 0.4774  44.777488 -0.830808   
5        const: 49.4582<br>x2: -1

In [ ]:
# Create a dictionary of dataframes
df_dict = {'Example OLS': df_example}

# Plotting DAGs for the residuals
plot_dags_for_residuals(df_dict, method='ols', weights=weights)